<a href="https://colab.research.google.com/github/onewon1234/AI_DL_Project/blob/KoElectra/KOelectra_train2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import itertools
import os

In [2]:
import os
os.getcwd()

'/content'

In [3]:
df = pd.read_csv("/content/train.csv")
df.head()

,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2


In [4]:
def preprocess_pairwise(df):
    data = []
    for _, row in df.iterrows():
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        answer = [row[f'answer_{i}'] for i in range(4)]
        ordered = [sentences[i] for i in answer]
        positive_pairs = [(ordered[i], ordered[i+1]) for i in range(3)]
        all_pairs = list(itertools.permutations(sentences, 2))
        for s1, s2 in all_pairs:
            label = 1 if (s1, s2) in positive_pairs else 0
            data.append({'sentence1': s1, 'sentence2': s2, 'label': label})
    return pd.DataFrame(data)

In [5]:
pairwise_df = preprocess_pairwise(df)
os.makedirs("../data", exist_ok=True)
pairwise_df.to_csv("../data/cleaned_pairwise.csv", index=False)
print("[✅] cleaned_pairwise.csv 저장 완료")

[✅] cleaned_pairwise.csv 저장 완료


In [6]:
import torch
import numpy as np
import pandas as pd

In [34]:
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

In [10]:
# 시드 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [22]:
# 데이터셋 클래스 정의
class SentenceOrderDataset(Dataset):
    def __init__(self, sentences1, sentences2, labels, tokenizer, max_length=128):
        self.sentences1 = sentences1
        self.sentences2 = sentences2
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sent1 = str(self.sentences1[idx])
        sent2 = str(self.sentences2[idx])

        encoding = self.tokenizer.encode_plus(
            sent1,
            sent2,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [19]:
def create_data_pairs(sentences, negative_ratio=1.0):
    """문장 쌍과 레이블 생성"""
    positive_pairs = []
    negative_pairs = []
    labels = []

    # 연속된 문장들에서 positive pair 생성
    for i in range(len(sentences)-1):
        positive_pairs.append((sentences[i], sentences[i+1]))

    # negative pair 생성 (무작위로 순서 섞기)
    for _ in range(int(len(positive_pairs) * negative_ratio)):
        idx1, idx2 = random.sample(range(len(sentences)), 2)
        if abs(idx1 - idx2) > 1:  # 연속되지 않은 문장만 선택
            negative_pairs.append((sentences[idx1], sentences[idx2]))
    all_pairs = positive_pairs + negative_pairs
    labels = [1] * len(positive_pairs) + [0] * len(negative_pairs)

    # 데이터 섞기
    combined = list(zip(all_pairs, labels))
    random.shuffle(combined)
    pairs, labels = zip(*combined)

    return [p[0] for p in pairs], [p[1] for p in pairs], labels

In [37]:
def train_model(model, train_loader, val_loader, device, num_epochs=3):
    optimizer = AdamW(model.parameters(), lr=2e-5)
    best_accuracy = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

In [38]:
model.eval()
val_correct = 0
val_total = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

train_accuracy = train_correct / train_total
val_accuracy = val_correct / val_total

print(f'Epoch {epoch+1}:')
print(f'Training Loss: {train_loss/len(train_loader):.4f}')
print(f'Training Accuracy: {train_accuracy:.4f}')
print(f'Validation Accuracy: {val_accuracy:.4f}')

if val_accuracy > best_accuracy:
    best_accuracy = val_accuracy
    torch.save(model.state_dict(), 'best_model.pt')

NameError: name 'model' is not defined

In [39]:
def main():
    # 시드 설정
    set_seed(42)

    # 예시 데이터 (실제 데이터로 교체 필요)
    sentences = [
        "블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.",
        "이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",
        "결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성할 잠재력을 지닌다.",
        "각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다."
    ]

    # 데이터 쌍 생성
    sent1_list, sent2_list, labels = create_data_pairs(sentences)

    # 토크나이저 및 모델 로드
    tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
    model = ElectraForSequenceClassification.from_pretrained(
        'monologg/koelectra-base-v3-discriminator',
        num_labels=2
    )

    # 데이터 분할
    train_sent1, val_sent1, train_sent2, val_sent2, train_labels, val_labels = train_test_split(
        sent1_list, sent2_list, labels, test_size=0.2, random_state=42
    )

    # 데이터셋 생성
    train_dataset = SentenceOrderDataset(train_sent1, train_sent2, train_labels, tokenizer)
    val_dataset = SentenceOrderDataset(val_sent1, val_sent2, val_labels, tokenizer)

    # 데이터로더 생성
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)

    # GPU 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # 모델 학습
    train_model(model, train_loader, val_loader, device)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

NameError: name 'AdamW' is not defined

In [40]:
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification

def load_model(model_path):
    """저장된 모델 로드"""
    model = ElectraForSequenceClassification.from_pretrained(
        'monologg/koelectra-base-v3-discriminator',
        num_labels=2
    )
    model.load_state_dict(torch.load(model_path))
    return model

def predict_order(sent1, sent2, model, tokenizer, device):
    """두 문장의 순서 예측"""
    model.eval()

    # 토크나이징
    encoding = tokenizer.encode_plus(
        sent1,
        sent2,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)

    return bool(predicted.item())

def main():
    # 모델과 토크나이저 로드
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = load_model('best_model.pt')
    model.to(device)
    tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')

    # 예시 문장
    sent1 = "작은 목표부터 시작하면 지속적인 동기를 유지할 수 있다."
    sent2 = "결국, 도덕적 공동체의 진정한 힘은 포용성과 다양성에서 비롯된다."

    # 순서 예측
    is_correct_order = predict_order(sent1, sent2, model, tokenizer, device)
    print(f"문장1: {sent1}")
    print(f"문장2: {sent2}")
    print(f"올바른 순서입니까? {'예' if is_correct_order else '아니오'}")

if __name__ == "__main__":
    main()

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pt'

In [ ]:
# 문장 순서 예측 모델 (KoELECTRA)

이 프로젝트는 KoELECTRA를 사용하여 두 문장 간의 순서를 예측하는 모델을 구현합니다.

## 설치 방법

1. 필요한 패키지 설치:
```bash
pip install -r requirements.txt
```

## 사용 방법

### 1. 모델 학습
```bash
python train_sentence_order.py
```
- 학습된 모델은 'best_model.pt'로 저장됩니다.
- 기본적으로 예시 데이터를 사용하여 학습합니다.
- 실제 데이터를 사용하려면 `train_sentence_order.py`의 `sentences` 리스트를 수정하세요.

### 2. 예측
```bash
python predict.py
```
- 저장된 모델을 사용하여 두 문장의 순서를 예측합니다.
- 예측하고 싶은 문장 쌍을 수정하려면 `predict.py`의 `sent1`과 `sent2`를 수정하세요.

## 모델 설명

- 사용 모델: monologg/koelectra-base-v3-discriminator
- 입력: 두 개의 연속된 문장
- 출력: 두 문장이 올바른 순서인지 여부 (이진 분류)
- 학습 방식:
  - Positive examples: 연속된 문장 쌍
  - Negative examples: 무작위로 선택된 비연속 문장 쌍

## 주의사항

- GPU가 있는 경우 자동으로 GPU를 사용합니다.
- 학습 데이터의 크기와 품질에 따라 모델의 성능이 달라질 수 있습니다.
- 실제 사용 시에는 충분한 양의 학습 데이터를 준비하는 것이 좋습니다.